In [1]:
import simpy
import random
import statistics
from dataclasses import dataclass, field

In [2]:
SEED = 42
random.seed(SEED)

# -------------------
# Parámetros globales
# -------------------
TIEMPO_REGISTRO_PROMEDIO = 2.0     # min
TIEMPO_CONSULTA_PROMEDIO = 7.0     # min
TASA_LLEGADA_PACIENTES   = 5.0     # min/paciente (interarrivo ~Exp con media=5)
TIEMPO_SIMULACION        = 120     # min

In [3]:
# -------------
# Monitoreadores
# -------------
@dataclass
class ResourceStats:
    env: simpy.Environment
    resource: simpy.Resource
    busy: int = 0
    last_t: float = 0.0
    busy_time_acc: float = 0.0
    q_len_acc: float = 0.0

    def update_time_weighted(self):
        dt = self.env.now - self.last_t
        # tiempo ponderado de ocupación (busy/ capacidad)
        self.busy_time_acc += dt * self.busy
        # longitud de cola promedio ponderada
        self.q_len_acc += dt * len(self.resource.queue)
        self.last_t = self.env.now

    def on_service_start(self):
        self.update_time_weighted()
        self.busy += 1

    def on_service_end(self):
        self.update_time_weighted()
        self.busy -= 1

    def utilization(self):
        cap = self.resource.capacity
        return (self.busy_time_acc / max(self.env.now, 1e-9)) / cap

    def avg_q_len(self):
        return self.q_len_acc / max(self.env.now, 1e-9)


@dataclass
class RunResults:
    n_pacientes: int
    t_total_prom: float
    t_total_max: float
    t_cola_reg_prom: float
    t_cola_cons_prom: float
    util_reg: float
    util_med: float
    q_reg_prom: float
    q_med_prom: float
    raw_totales: list = field(default_factory=list)

Proceso del paciente y generador

In [4]:
def paciente(env, nombre, recep, med, stats_recep, stats_med,
             tiempos_totales, colas_reg, colas_cons):
    t_llegada = env.now

    # --- Registro ---
    t_req = env.now
    with recep.request() as req:
        yield req
        t_wait = env.now - t_req
        colas_reg.append(t_wait)

        stats_recep.on_service_start()
        t_serv = random.expovariate(1.0 / TIEMPO_REGISTRO_PROMEDIO)
        yield env.timeout(t_serv)
        stats_recep.on_service_end()

    # --- Consulta ---
    t_req = env.now
    with med.request() as req:
        yield req
        t_wait = env.now - t_req
        colas_cons.append(t_wait)

        stats_med.on_service_start()
        t_serv = random.expovariate(1.0 / TIEMPO_CONSULTA_PROMEDIO)
        yield env.timeout(t_serv)
        stats_med.on_service_end()

    tiempos_totales.append(env.now - t_llegada)


def generador_pacientes(env, recep, med, stats_recep, stats_med,
                        tiempos_totales, colas_reg, colas_cons):
    i = 0
    while True:
        # interarrivo ~Exp(media = TASA_LLEGADA_PACIENTES)
        yield env.timeout(random.expovariate(1.0 / TASA_LLEGADA_PACIENTES))
        i += 1
        env.process(paciente(env, f"Paciente {i}", recep, med, stats_recep, stats_med,
                             tiempos_totales, colas_reg, colas_cons))


Función para correr escenarios y recolectar métricas

In [5]:
def correr_escenario(num_recepcionistas=1, num_medicos=2, tiempo=TIEMPO_SIMULACION, seed=SEED):
    random.seed(seed)
    env = simpy.Environment()
    recep = simpy.Resource(env, capacity=num_recepcionistas)
    med   = simpy.Resource(env, capacity=num_medicos)

    stats_recep = ResourceStats(env, recep)
    stats_med   = ResourceStats(env, med)

    tiempos_totales = []
    colas_reg = []
    colas_cons = []

    env.process(generador_pacientes(env, recep, med, stats_recep, stats_med,
                                    tiempos_totales, colas_reg, colas_cons))
    env.run(until=tiempo)

    n = len(tiempos_totales)
    t_prom = statistics.mean(tiempos_totales) if n else 0.0
    t_max  = max(tiempos_totales) if n else 0.0
    c_reg_prom  = statistics.mean(colas_reg) if colas_reg else 0.0
    c_cons_prom = statistics.mean(colas_cons) if colas_cons else 0.0

    res = RunResults(
        n_pacientes=n,
        t_total_prom=t_prom,
        t_total_max=t_max,
        t_cola_reg_prom=c_reg_prom,
        t_cola_cons_prom=c_cons_prom,
        util_reg=stats_recep.utilization(),
        util_med=stats_med.utilization(),
        q_reg_prom=stats_recep.avg_q_len(),
        q_med_prom=stats_med.avg_q_len(),
        raw_totales=tiempos_totales
    )
    return res

def imprimir_res(nombre, R):
    print(f"\n=== {nombre} ===")
    print(f"Pacientes atendidos            : {R.n_pacientes}")
    print(f"Tiempo total PROM (min)        : {R.t_total_prom:.2f}")
    print(f"Tiempo total MAX  (min)        : {R.t_total_max:.2f}")
    print(f"Cola REG PROM (min)            : {R.t_cola_reg_prom:.2f}")
    print(f"Cola CONS PROM (min)           : {R.t_cola_cons_prom:.2f}")
    print(f"Utilización REG                : {R.util_reg:.2%}")
    print(f"Utilización MED                : {R.util_med:.2%}")
    print(f"Long. cola REG PROM            : {R.q_reg_prom:.2f}")
    print(f"Long. cola MED PROM            : {R.q_med_prom:.2f}")


Experimentos que pide el lab

In [6]:
# Baseline: 1 recepcionista, 2 médicos
base = correr_escenario(num_recepcionistas=1, num_medicos=2)
imprimir_res("Escenario Base (R=1, M=2)", base)

# Pregunta 2: aumentar médicos a 5
m5 = correr_escenario(num_recepcionistas=1, num_medicos=5)
imprimir_res("Médicos=5 (R=1, M=5)", m5)

# Pregunta 3: volver M=2 y subir recepcionistas a 2
r2 = correr_escenario(num_recepcionistas=2, num_medicos=2)
imprimir_res("Recepcionistas=2 (R=2, M=2)", r2)

# Mini-cuadro comparativo para el reporte
import pandas as pd
df = pd.DataFrame([
    {"Escenario":"Base R=1 M=2", "Pacientes":base.n_pacientes,
     "T_prom":base.t_total_prom, "T_max":base.t_total_max,
     "Cola_reg":base.t_cola_reg_prom, "Cola_cons":base.t_cola_cons_prom,
     "Util_reg":base.util_reg, "Util_med":base.util_med},
    {"Escenario":"R=1 M=5", "Pacientes":m5.n_pacientes,
     "T_prom":m5.t_total_prom, "T_max":m5.t_total_max,
     "Cola_reg":m5.t_cola_reg_prom, "Cola_cons":m5.t_cola_cons_prom,
     "Util_reg":m5.util_reg, "Util_med":m5.util_med},
    {"Escenario":"R=2 M=2", "Pacientes":r2.n_pacientes,
     "T_prom":r2.t_total_prom, "T_max":r2.t_total_max,
     "Cola_reg":r2.t_cola_reg_prom, "Cola_cons":r2.t_cola_cons_prom,
     "Util_reg":r2.util_reg, "Util_med":r2.util_med},
])
df.style.format({
    "T_prom":"{:.2f}", "T_max":"{:.2f}",
    "Cola_reg":"{:.2f}", "Cola_cons":"{:.2f}",
    "Util_reg":"{:.0%}", "Util_med":"{:.0%}"
})



=== Escenario Base (R=1, M=2) ===
Pacientes atendidos            : 23
Tiempo total PROM (min)        : 10.82
Tiempo total MAX  (min)        : 29.28
Cola REG PROM (min)            : 0.81
Cola CONS PROM (min)           : 2.25
Utilización REG                : 28.64%
Utilización MED                : 59.89%
Long. cola REG PROM            : 0.23
Long. cola MED PROM            : 0.95

=== Médicos=5 (R=1, M=5) ===
Pacientes atendidos            : 26
Tiempo total PROM (min)        : 9.02
Tiempo total MAX  (min)        : 28.34
Cola REG PROM (min)            : 1.28
Cola CONS PROM (min)           : 0.00
Utilización REG                : 41.44%
Utilización MED                : 25.22%
Long. cola REG PROM            : 0.41
Long. cola MED PROM            : 0.00

=== Recepcionistas=2 (R=2, M=2) ===
Pacientes atendidos            : 22
Tiempo total PROM (min)        : 11.11
Tiempo total MAX  (min)        : 29.28
Cola REG PROM (min)            : 0.00
Cola CONS PROM (min)           : 2.71
Utilización REG  

,Escenario,Pacientes,T_prom,T_max,Cola_reg,Cola_cons,Util_reg,Util_med
0,Base R=1 M=2,23,10.82,29.28,0.81,2.25,29%,60%
1,R=1 M=5,26,9.02,28.34,1.28,0.00,41%,25%
2,R=2 M=2,22,11.11,29.28,0.00,2.71,11%,66%


### Pregunta 1
Escenario base (R=1, M=2)

Resultados que se obtuvieron:

23 pacientes atendidos

Tiempo total promedio: 10.82 min

Tiempo total máximo: 29.28 min

Utilización recepcionista: ≈29%

Utilización médicos: ≈60%

### Conclusión:

El sistema funciona de forma aceptable, pero ya se observa carga alta en médicos.

El tiempo promedio de espera total es moderado (≈11 min), y la etapa más lenta es claramente la consulta.

Cuello de botella: Médicos (mayor utilización, mayor cola y espera).






### Pregunta 2

Escenario con más médicos (R=1, M=5)

Resultados:

Pacientes: 26

Promedio total: 9.02 min

Cola de consulta: 0.00 min

Utilización médicos: 25%

Utilización recepción: 41%

### Conclusión:
Al aumentar los médicos, el tiempo promedio baja de 10.82 → 9.02 min (mejora significativa).

Las colas de consulta desaparecen y el servicio se vuelve fluido, lo que confirma empíricamente que el cuello de botella era la consulta médica.

Interpretación: el sistema ahora está limitado por la recepción.


### Pregunta 3
Escenario con más recepcionistas (R=2, M=2)

Resultados:

Pacientes: 22

Promedio total: 11.11 min

Cola de registro: 0.00 min

Cola de consulta: 2.71 min

Utilización médicos: 66%

### Conclusión:

Incrementar recepcionistas no mejora el rendimiento general; el promedio incluso sube levemente.

Los médicos siguen siendo el recurso crítico (se mantienen ocupados más del 60% del tiempo).

Interpretación: duplicar recepcionistas tiene efecto mínimo, ya que su etapa no era el cuello de botella.

### Recomendacion 

Recomendación profesional (síntesis ejecutiva)
| Escenario | Tiempo Promedio (min) | Cuello principal | Mejora global |
| --------- | --------------------- | ---------------- | ------------- |
| R=1 M=2   | 10.82                 | Médicos          | Base          |
| R=1 M=5   | 9.02                  | Recepción        |  Mejor       |
| R=2 M=2   | 11.11                 | Médicos          |  Peor        |


El análisis de simulación muestra que los médicos constituyen el cuello de botella del sistema.
Aumentar su cantidad reduce significativamente el tiempo promedio total (de 10.82 a 9.02 min), mientras que aumentar recepcionistas no genera mejoras sustanciales.
Por tanto, la inversión prioritaria debe dirigirse a incrementar el número de médicos, pues ofrece el mayor impacto en tiempos de espera.
Solo si el costo lo permite, podría evaluarse un equilibrio posterior en la etapa de recepción.